<a href="https://colab.research.google.com/github/arthur-kt/PPP-Intertemporal-Converter/blob/main/PPP_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PPP Interporal Converter

## Parâmetros Iniciais

In [62]:
!pip install fredapi
!pip install pycountry

from fredapi import Fred
import pycountry
import pandas as pd
from datetime import datetime, timedelta
import requests
from collections import defaultdict

# Inicialize a chave do FRED
fred = Fred(api_key='11cb1e53b57458acaddd29a16e0efe1b')


# Obtenha o CPI dos EUA para ajustar pela inflação
cpi_data = fred.get_series('CPIAUCSL')
cpi_df = cpi_data.to_frame(name='CPI')
cpi_df.index = pd.to_datetime(cpi_df.index)
cpi_df_ano = cpi_df.resample('YE').mean()  # Média anual do CPI

# CPI atual para trazer o valor a 2024
cpi_atual = cpi_df_ano.loc['2024'].values[0]

## Importar dados PPP

In [63]:
# Importar dados PPP
url = 'https://raw.githubusercontent.com/arthur-kt/PPP-Intertemporal-Converter/eabb3ca9caaca37dc9d9f1ae46ad03f7bc699954/ppp_att.csv'
ppp_data =  pd.read_csv(url, delimiter=',')

## Obter taxa de conversão do dólar 2024

In [64]:
# Função para obter o código de moeda usando o código de país ISO2
def get_currency_code(country_code):
    try:
        country = pycountry.countries.get(alpha_2=country_code)
        currency = pycountry.currencies.get(numeric=country.numeric)
        return currency.alpha_3 if currency else None
    except AttributeError:
        print(f"Não foi possível encontrar a moeda para o código de país: {country_code}")
        return None

# Mudar ISO3 para ISO2
def iso3_to_iso2(iso3):
    try:
    # Convert iso3 to string and ensure it's uppercase
        iso3 = str(iso3).upper()
        return pycountry.countries.get(alpha_3=iso3).alpha_2
    except (LookupError, AttributeError):
        # Handle both LookupError and AttributeError
        return None

# Importar taxa de câmbio dos países
url2 = 'https://raw.githubusercontent.com/arthur-kt/PPP-Intertemporal-Converter/refs/heads/main/EXCHANGE_RATES.csv'
exchange_rates24 =  pd.read_csv(url2)
exchange_rates24 = exchange_rates24[['Country Code', '2024 [2024]']]
exchange_rates24 = exchange_rates24.rename(columns={'2024 [2024]': '2024'})
exchange_rates24['Country Code'] = exchange_rates24['Country Code'].apply(iso3_to_iso2)

## Função Conversão

In [71]:
# Função para obter o PPP ajustado de um país em um ano específico a partir do CSV
def get_ppp_adjusted(country_code, year):
    ppp_row = ppp_data[(ppp_data['Country ID'] == country_code) & (ppp_data['Year'] == year)]
    return ppp_row.iloc[0]['PPP']

# Função para converter gasto local em dólar ajustado pela inflação dos EUA
def converter_para_dolar_corrente(row):
    # Para o ano 2023, usa os dados de PPP externos
    if row['Ano2'] == 0:
        if row['Ano'] == 2024:
            # Procura o exchange rate diretamente no dataframe exchange_rates24
            exchange_rate_row = exchange_rates24[exchange_rates24['Country Code'] == row['País_Moeda']]
            if not exchange_rate_row.empty:
                exchange_rate = exchange_rate_row.iloc[0]['2024']  # Obtém a taxa de câmbio para 2024
                exchange_rate = float(exchange_rate)
                gasto_em_dolar_2024 = row['Gasto_Local'] / exchange_rate
                return gasto_em_dolar_2024
        else:
            # Para demais anos
            ppp_adjusted = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
            gasto_em_dolar_ppp = row['Gasto_Local'] / ppp_adjusted

            # Ajuste pela inflação dos EUA
            cpi_ano_referencia = cpi_df_ano.loc[str(row['Ano'])].values[0]
            gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_ano_referencia)

    elif row['Ano2'] == 2024:
        # Calcular gasto via taxa de câmbio de 2024
        exchange_rate_row = exchange_rates24[exchange_rates24['Country Code'] == row['País_Moeda']]
        if not exchange_rate_row.empty:
            exchange_rate = float(exchange_rate_row.iloc[0]['2024'])
            gasto_em_dolar_2024 = row['Gasto_Local'] / exchange_rate

            # Calcular também via PPP + inflação, considerando apenas Ano1
            ppp_ano1 = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
            gasto_em_dolar_ppp = row['Gasto_Local'] / ppp_ano1

            cpi_ano1 = cpi_df_ano.loc[str(row['Ano'])].values[0]
            gasto_ajustado_ppp = gasto_em_dolar_ppp * (cpi_atual / cpi_ano1)

            # Média entre os dois métodos
            gasto_ajustado = (gasto_em_dolar_2024 + gasto_ajustado_ppp) / 2

    else:
        # Caso exista Ano2, calcula o PPP médio entre Ano1 e Ano2
        ppp_ano1 = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
        ppp_ano2 = get_ppp_adjusted(row['País_Moeda'], row['Ano2'])

        # Média dos gastos entre Ano1 e Ano2
        gasto_em_dolar_ppp = row['Gasto_Local'] / ((ppp_ano1 + ppp_ano2) / 2)

        # Ajuste pela inflação dos EUA usando CPI de Ano1 e Ano2
        cpi_ano1 = cpi_df_ano.loc[str(row['Ano'])].values[0]
        cpi_ano2 = cpi_df_ano.loc[str(row['Ano2'])].values[0]

        # Calcula a média do CPI para ajuste pela inflação
        cpi_media = (cpi_ano1 + cpi_ano2) / 2

        # Ajusta o gasto usando a média do CPI
        gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_media)

    return float(gasto_ajustado)


def agrupar_gastos(df):
    # Agrupa por 'Período' e 'País do Programa' e soma os valores de 'Gasto_Convertido_Dolar_2024'
    df_agrupado = df.groupby(['Período', 'País do Programa'])['Gasto_Convertido_Dolar_2024'].sum().reset_index()
    return df_agrupado

def agrupar_gastos_pais(df):
    # Agrupa por 'Período' e 'País do Programa' e soma os valores de 'Gasto_Convertido_Dolar_2024'
    df_pais = df.groupby(['País do Programa'])['Gasto_Convertido_Dolar_2024'].sum().reset_index()
    return df_pais

def agrupar_gastos_programa(df):
    # Agrupa por 'Período' e 'País do Programa' e soma os valores de 'Gasto_Convertido_Dolar_2024'
    df_prog = df.groupby(['Programa', 'País do Programa'])['Gasto_Convertido_Dolar_2024'].sum().reset_index()
    return df_prog

## Função Formatação

In [66]:
# Função para formatar valores em milhões, bilhões ou trilhões para Moeda Local
def formatar_valor_moeda_local(valor, codigo_pais):
    currency_code = get_currency_code(codigo_pais)  # Obtém o código da moeda do país
    if not currency_code:
        currency_code = "Moeda desconhecida"  # Se não houver código, exibe "Moeda desconhecida"

    if valor >= 1_000_000_000_000:  # Trilhão
        return f"{currency_code} {valor / 1_000_000_000_000:.2f} trilhões"
    elif valor >= 1_000_000_000:  # Bilhão
        return f"{currency_code} {valor / 1_000_000_000:.2f} bilhões"
    elif valor >= 1_000_000:  # Milhão
        return f"{currency_code} {valor / 1_000_000:.2f} milhões"
    else:
        return f"{currency_code} {valor:.2f}"

# Função para formatar valores em milhões, bilhões ou trilhões para Dólar Ajustado
def formatar_valor_dolar(valor):
    if valor >= 1_000_000_000_000:  # Trilhão
        return f"USD {valor / 1_000_000_000_000:.2f} trilhões"
    elif valor >= 1_000_000_000:  # Bilhão
        return f"USD {valor / 1_000_000_000:.2f} bilhões"
    elif valor >= 1_000_000:  # Milhão
        return f"USD {valor / 1_000_000:.2f} milhões"
    else:
        return f"USD {valor:.2f}"

# Função para juntar Ano com Ano2
def definir_periodo(df):
    # Verifica se as colunas Ano e Ano2 existem no DataFrame
    if 'Ano' in df.columns and 'Ano2' in df.columns:
        # Aplica a condição para criar a coluna Período
        df['Período'] = df.apply(lambda row: str(row['Ano']) if row['Ano2'] == 0 else f"{row['Ano']} à {row['Ano2']}", axis=1)
    else:
        print("Colunas 'Ano' e 'Ano2' não encontradas no DataFrame.")
    return df

## Insira seus dados:

In [74]:
# Dados

## Insira em 'País_Moeda' o o país emissor da moeda utilizada (no caso de uso de moedas diferentes do país de origem), em 'País do Programa' o país de origem do Programa, em 'Ano' o ano da origem dos recursos, em 'Gasto_Local' a quantia dispendida, em 'Programa' o nome do Programa e em 'Descrição' uma breve descrição da função ou destinação dos valores.
## A coluna 'Ano2' serve para os programas que tem custos entre 2 diferentes anos, sendo necessário calcular a média do PPP e da inflação americana. Então, insira no 'Ano2' o último ano do programa, se não, preencha com 0.

dados = {
    'País_Moeda': [
        'US', 'GB', 'GB', 'KR', 'IN', 'IN', 'IN', 'IN', 'IN', 'IN', 'IN',
        'IN', 'IN', 'IN', 'IN', 'IN', 'IN', 'IN', 'ZA', 'MX', 'MX', 'MX', 'MX'
    ],
    'País do Programa': [
        'US', 'GB', 'GB', 'KR', 'IN', 'IN', 'IN', 'IN', 'IN', 'IN', 'IN',
        'IN', 'IN', 'IN', 'IN', 'IN', 'IN', 'IN', 'ZA', 'MX', 'MX', 'MX', 'MX'
    ],
    'Ano': [
        2023, 2017, 2024, 2011, 2019, 2023, 2022, 2023, 2014, 2009, 2023,
        2017, 2018, 2019, 2020, 2021, 2022, 2023, 2020, 2020, 2021, 2022, 2023
    ],
    'Ano2': [
        0, 2021, 0, 2020, 0, 0, 0, 0, 0, 2024, 0,
        2018, 2019, 2020, 2021, 2022, 2023, 2024, 0, 0, 0, 0, 2024
    ],
    'Gasto_Local': [
        1000000000,
        255000000,
        800000000,
        1000000000,
        4800000000,
        547300000,
        215000000,
        50000000000,
        5970000000,
        5745500000,
        300000000,
        30119780,
        56422100,
        64280438,
        56644734,
        92100000,
        61300000,
        65000000,
        4000000,
        74524500000,
        48513400000,
        40244300000,
        27702000000
    ],
    'Programa': [
        'Bold Goals for U.S. Biotechnology and Biomanufacturing',
        'Medicines Manufacturing Challenge (2017)',
        'Advanced Research and Invention Agency (ARIA) (2020)',
        'Korea Drug Development Fund (KDDF)',
        'Scheme for Development of Pharmaceutical Industry (2019)',
        'Assistance to Pharmaceutical Industry for Common Facilities (APICF)',
        'Pharmaceutical & Medical Devices Promotion and Development Scheme (PMPDS)',
        'National Policy on R&D and Innovation in the Pharma-MedTech Sector (2023)',
        'Human Resource Development Scheme for Health Research (2013, renovado)',
        'Biotechnology Industry Partnership Programme (BIPP)',
        'BIRAC SEED Fund',
        'BIRAC SPARSH (2017-2018)',
        'BIRAC SPARSH (2018-2019)',
        'BIRAC SPARSH (2019-2020)',
        'BIRAC SPARSH (2020-2021)',
        'BIRAC SPARSH (2021-2022)',
        'BIRAC SPARSH (2022-2023)',
        'BIRAC SPARSH (2023-2024)',
        'Strategic Health Innovation Partnership',
        'Programa Sectorial de Salud 2020-2024 (2020)',
        'Programa Sectorial de Salud 2020-2024 (2021)',
        'Programa Sectorial de Salud 2020-2024 (2022)',
        'Programa Sectorial de Salud 2020-2024 (2023–jun/2024)',
    ],
    'Descrição': [
        'Destinado pelo Departamento de Defesa dos EUA para apoiar a biotecnologia e a biomanufatura, impulsionando a inovação, a bioeconomia e a produção de biocombustíveis e bioprodutos renováveis.',
        'Investimento no desenvolvimento de tecnologias inovadoras para fabricar medicamentos e acelerar o acesso dos pacientes a novos medicamentos e tratamentos',
        'Alocação inicial ao longo de cinco anos até 2025/26',
        'Recursos ao longo de 9 anos para financiar o desenvolvimento de mais de 100 medicamentos promissores na Coreia, oferecendo investimento e suporte empresarial',
        'Orçamento destinado a apoiar a indústria farmacêutica nacional, com foco em centros de facilidades comuns, atualização tecnológica e promoção do setor.',
        'Montante total investido em três projetos para fortalecer os clusters de MSME farmacêuticas existentes',
        'Desembolso financeiro entre 2021/2022 e 2025/2026',
        'Sob essa política, foi lançado o esquema Promotion of Research and Innovation in Pharma and MedTech Sector (PRIP), com este orçamento.',
        'Custo estimado total',
        'Fundos comprometidos entre 2009 - 2024',
        'SEED Fund apoiou 112 startups com esse investimento de equity total',
        'Desembolsos totais feitos pelo programa SPARSH durante os respectivos anos fiscais: 2017-2018',
        'Desembolsos totais feitos pelo programa SPARSH durante os respectivos anos fiscais: 2018-2019',
        'Desembolsos totais feitos pelo programa SPARSH durante os respectivos anos fiscais: 2019-2020',
        'Desembolsos totais feitos pelo programa SPARSH durante os respectivos anos fiscais: 2020-2021',
        'Desembolsos totais feitos pelo programa SPARSH durante os respectivos anos fiscais: 2021-2022',
        'Desembolsos totais feitos pelo programa SPARSH durante os respectivos anos fiscais: 2022-2023',
        'Desembolsos totais feitos pelo programa SPARSH durante os respectivos anos fiscais: 2023-2024',
        'Financiamento inicial (seed funding) para apoiar projetos de pesquisa em saúde nas quatro principais instituições de pesquisa pública da província de Eastern Cape',
        'Investimento governamental diretamente executado no âmbito do programa. com foco no sistema de saúde como um todo (impactando a industria da saúde): 2020',
        'Investimento governamental diretamente executado no âmbito do programa. com foco no sistema de saúde como um todo (impactando a industria da saúde): 2021',
        'Investimento governamental diretamente executado no âmbito do programa. com foco no sistema de saúde como um todo (impactando a industria da saúde): 2022',
        'Investimento governamental diretamente executado no âmbito do programa. com foco no sistema de saúde como um todo (impactando a industria da saúde): 2023 – junho/2024',
    ]
}


#Check se todos dados estão com o mesmo número de entradas
print(len(dados['Programa']),
len(dados['Descrição']),
len(dados['País_Moeda']),
len(dados['País do Programa']),
len(dados['Ano']),
len(dados['Ano2']),
len(dados['Gasto_Local']))

df = pd.DataFrame(dados)

23 23 23 23 23 23 23


## Conversor

In [75]:
# Aplicar a função para cada linha no DataFrame
df['Gasto_Convertido_Dolar_2024'] = df.apply(converter_para_dolar_corrente, axis=1)

# Aplicar Função para juntar Ano 1 e 2
df = definir_periodo(df)

# Aplicar as novas funções para as colunas 'Gasto_Local' e 'Gasto_Convertido_Dolar_2024'
df['Gasto Local'] = df.apply(lambda row: formatar_valor_moeda_local(row['Gasto_Local'], row['País_Moeda']), axis=1)
df['Gasto Convertido US$ 2024'] = df['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)

# Exibir o DataFrame com as colunas formatadas
df[['País do Programa', 'Programa', 'Descrição', 'Ano', 'Ano2', 'Período', 'Gasto Local', 'Gasto Convertido US$ 2024']]

<ipython-input-71-c21d071803b3>:62: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(gasto_ajustado)


,País do Programa,Programa,Descrição,Ano,Ano2,Período,Gasto Local,Gasto Convertido US$ 2024
0,US,Bold Goals for U.S. Biotechnology and Biomanuf...,Destinado pelo Departamento de Defesa dos EUA ...,2023,0,2023,USD 1.00 bilhões,USD 1.03 bilhões
1,GB,Medicines Manufacturing Challenge (2017),Investimento no desenvolvimento de tecnologias...,2017,2021,2017 à 2021,GBP 255.00 milhões,USD 466.34 milhões
2,GB,Advanced Research and Invention Agency (ARIA) ...,Alocação inicial ao longo de cinco anos até 20...,2024,0,2024,GBP 800.00 milhões,USD 1.02 bilhões
3,KR,Korea Drug Development Fund (KDDF),Recursos ao longo de 9 anos para financiar o d...,2011,2020,2011 à 2020,KRW 1.00 bilhões,USD 1.54 milhões
4,IN,Scheme for Development of Pharmaceutical Indus...,Orçamento destinado a apoiar a indústria farma...,2019,0,2019,INR 4.80 bilhões,USD 291.01 milhões
5,IN,Assistance to Pharmaceutical Industry for Comm...,Montante total investido em três projetos para...,2023,0,2023,INR 547.30 milhões,USD 27.87 milhões
6,IN,Pharmaceutical & Medical Devices Promotion and...,Desembolso financeiro entre 2021/2022 e 2025/2026,2022,0,2022,INR 215.00 milhões,USD 11.15 milhões
7,IN,National Policy on R&D and Innovation in the P...,"Sob essa política, foi lançado o esquema Promo...",2023,0,2023,INR 50.00 bilhões,USD 2.55 bilhões
8,IN,Human Resource Development Scheme for Health R...,Custo estimado total,2014,0,2014,INR 5.97 bilhões,USD 432.31 milhões
9,IN,Biotechnology Industry Partnership Programme (...,Fundos comprometidos entre 2009 - 2024,2009,2024,2009 à 2024,INR 5.75 bilhões,USD 348.56 milhões


In [76]:
df_agrupado = agrupar_gastos(df)

# Convert 'Gasto_Convertido_Dolar_2024' to numeric before formatting
df_agrupado['Gasto_Convertido_Dolar_2024'] = pd.to_numeric(df_agrupado['Gasto_Convertido_Dolar_2024'], errors='coerce')

# Apply formatting only once
df_agrupado['Gasto_Convertido_Dolar_2024'] = df_agrupado['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)

df_agrupado

,Período,País do Programa,Gasto_Convertido_Dolar_2024
0,2009 à 2024,IN,USD 348.56 milhões
1,2011 à 2020,KR,USD 1.54 milhões
2,2014,IN,USD 432.31 milhões
3,2017 à 2018,IN,USD 1.86 milhões
4,2017 à 2021,GB,USD 466.34 milhões
5,2018 à 2019,IN,USD 3.43 milhões
6,2019,IN,USD 291.01 milhões
7,2019 à 2020,IN,USD 3.87 milhões
8,2020,MX,USD 9.20 bilhões
9,2020,ZA,USD 668713.56


In [77]:
df_pais = agrupar_gastos_pais(df)

# Convert 'Gasto_Convertido_Dolar_2024' to numeric before formatting
df_pais['Gasto_Convertido_Dolar_2024'] = pd.to_numeric(df_pais['Gasto_Convertido_Dolar_2024'], errors='coerce')

# Apply formatting only once
df_pais['Gasto_Convertido_Dolar_2024'] = df_pais['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)

df_pais

,País do Programa,Gasto_Convertido_Dolar_2024
0,GB,USD 1.49 bilhões
1,IN,USD 3.69 bilhões
2,KR,USD 1.54 milhões
3,MX,USD 21.54 bilhões
4,US,USD 1.03 bilhões
5,ZA,USD 668713.56


In [78]:
df_prog = agrupar_gastos_programa(df)

# Convert 'Gasto_Convertido_Dolar_2024' to numeric before formatting
df_prog['Gasto_Convertido_Dolar_2024'] = pd.to_numeric(df_prog['Gasto_Convertido_Dolar_2024'], errors='coerce')

# Apply formatting only once
df_prog['Gasto_Convertido_Dolar_2024'] = df_prog['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)

df_prog

,Programa,País do Programa,Gasto_Convertido_Dolar_2024
0,Advanced Research and Invention Agency (ARIA) ...,GB,USD 1.02 bilhões
1,Assistance to Pharmaceutical Industry for Comm...,IN,USD 27.87 milhões
2,BIRAC SEED Fund,IN,USD 15.28 milhões
3,BIRAC SPARSH (2017-2018),IN,USD 1.86 milhões
4,BIRAC SPARSH (2018-2019),IN,USD 3.43 milhões
5,BIRAC SPARSH (2019-2020),IN,USD 3.87 milhões
6,BIRAC SPARSH (2020-2021),IN,USD 3.27 milhões
7,BIRAC SPARSH (2021-2022),IN,USD 4.95 milhões
8,BIRAC SPARSH (2022-2023),IN,USD 3.15 milhões
9,BIRAC SPARSH (2023-2024),IN,USD 2.04 milhões
